In [1]:
import pandas as pd

In [2]:
corpus = pd.read_csv('./corpus (1).txt')

In [3]:
movies = corpus[corpus['domain'] == 'movies']
movies = movies.drop(columns=['Unnamed: 0', 'domain', 'author', 'likes', 'unlikes', 'date', 'collect_date', 
                              'replies', 'favorite', 'want_see', 'recommend', 'see', 'internal_id'])
movies = movies.dropna(subset=['text'])
movies = movies.dropna(subset=['stars'])

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1838895 entries, 0 to 1839850
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   object  object 
 1   text    object 
 2   stars   float64
dtypes: float64(1), object(2)
memory usage: 56.1+ MB


In [5]:
def classify_stars(stars):
    if 0 <= stars < 1.5:
        return 1
    elif 1.5 <= stars < 2.5:
        return 2
    elif 2.5 <= stars < 3.5:
        return 3
    elif 3.5 <= stars < 4.5:
        return 4
    elif stars >= 4.5:
        return 5
    else:
        return None
    
#movies['discrete_stars'] = movies['stars']
movies['discrete_stars'] = movies['stars'].apply(classify_stars)

In [6]:
movies['discrete_stars'].value_counts(normalize=True)*100

discrete_stars
5    39.446026
4    26.368281
1    20.220676
3    10.576080
2     3.388937
Name: proportion, dtype: float64

In [7]:
# Para cada valor de discrete_stars, selecionar 62360 amostras aleatórias
least_amount = movies['discrete_stars'].value_counts().min()
sampled_movies = movies.groupby('discrete_stars').apply(lambda x: x.sample(n=least_amount, random_state=42)).reset_index(drop=True)

C:\Users\lucca\AppData\Local\Temp\ipykernel_7532\3932032607.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_movies = movies.groupby('discrete_stars').apply(lambda x: x.sample(n=least_amount, random_state=42)).reset_index(drop=True)


In [8]:
sampled_movies['discrete_stars'].value_counts()

discrete_stars
1    62319
2    62319
3    62319
4    62319
5    62319
Name: count, dtype: int64

In [9]:
# Reduzir o tamanho do dataset para 50k amostras por classe
sampled_movies = sampled_movies.groupby('discrete_stars').apply(lambda x: x.sample(n=10000, random_state=42)).reset_index(drop=True)
sampled_movies['discrete_stars'].value_counts()

C:\Users\lucca\AppData\Local\Temp\ipykernel_7532\4023045877.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_movies = sampled_movies.groupby('discrete_stars').apply(lambda x: x.sample(n=10000, random_state=42)).reset_index(drop=True)


discrete_stars
1    10000
2    10000
3    10000
4    10000
5    10000
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

unique_movies = sampled_movies.groupby('object')['discrete_stars'].agg(lambda x: x.mode().iloc[0]).reset_index()

movies_train, movies_test = train_test_split(
    unique_movies,
    test_size=0.2,
    stratify=unique_movies['discrete_stars'],
    random_state=42
)

train_df = sampled_movies[sampled_movies['object'].isin(movies_train['object'])]
test_df = sampled_movies[sampled_movies['object'].isin(movies_test['object'])]

print("Treino:")
print(train_df['discrete_stars'].value_counts())
print()
print("Teste:")
print(test_df['discrete_stars'].value_counts())

Treino:
discrete_stars
2    8182
4    8161
3    8135
1    8071
5    8039
Name: count, dtype: int64

Teste:
discrete_stars
5    1961
1    1929
3    1865
4    1839
2    1818
Name: count, dtype: int64


In [11]:
train_df.shape[0], test_df.shape[0]

(40588, 9412)

In [12]:
train_df.shape[0] / sampled_movies.shape[0], test_df.shape[0] / sampled_movies.shape[0]

(0.81176, 0.18824)

In [13]:
train_df['object'].nunique(), test_df['object'].nunique()

(2043, 511)

In [14]:
train_df.head()

,object,text,stars,discrete_stars
1,namorados-para-sempre-t15562,fiquei mal quando terminei de ver,0.0,1
4,mortal-kombat-t15331,http://www.youtube.com/watch?v=hSCB5mBpxq0,0.0,1
5,idiocracia-t8163,"Concordo com o bruno. O filme é horrível, apes...",1.0,1
6,sweeney-todd-o-barbeiro-demoniaco-da-rua-fleet...,Excelente!! Adorei o musical e ver no filme fo...,0.0,1
7,anticristo-t8198,"Amiga, vamos melhorar.",1.0,1


In [15]:
# Exportar os datasets
train_df.to_csv('train_movies.csv', index=False)
test_df.to_csv('test_movies.csv', index=False)